# Coursework Part 1: Detecting Spam with Spark

These are the tasks for IN432 Big Data coursework 2019, part 1.  

This coursework is about classification of e-mail messages as spam or non-spam in Spark. We will go through the whole process from loading and preprocessing to training and testing classifiers in a distributed way in Spark. We wil use the techniques shown in the lextures and labs. I will also introduce here a few additional elements, such as the NLTK and some of the preprocessing and machine learning functions that come with Spark. You are not expected to need anything beyond the material handed out so far and in some cases the Spark documentation, to which I have put links in this document.  

The structure is similar to the lab sheets. I provide a code structure with gaps that you are supposed to file. In addition you should run 2 small experiments and comment on the results. The lines where you are supposed to add code or take another action are marked with ">>>" 
please leave the ">>>" in the text, comment out that line, and write your own code in the next line using a copy of that line as a starting point.

I have added numerous comments in text cells and the code cells to guid you through the program. Please read them carefully and ask if anything is unclear. 

Once you have completed the tasks, don't delete the outpus, but downlaod the notebook (outputs will be included) and upload it into the coursework submission area on Moodle. The coursework part counts for 50% or the total coursework.

## Load and prepare the data

We will use the lingspam dataset in this coursework (see [http://csmining.org/index.php/ling-spam-datasets.html](http://csmining.org/index.php/ling-spam-datasets.html) for more information).

The next cells only prepare the machine, as usual.

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install pyspark

import pyspark
sc = pyspark.SparkContext.getOrCreate()

    100% |████████████████████████████████| 213.4MB 79kB/s 
    100% |████████████████████████████████| 204kB 27.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/cd/54/c2/abfcc942eddeaa7101228ebd6127a30dbdf903c72db4235b23
Successfully built pyspark


In [0]:
# We have a new dataset in directory BigData/data/lingspam_public .
%cd /content/drive/My Drive/BigData/data/lingspam_public 
# the line above should output should show "bare  lemm  lemm_stop  readme.txt  stop"
!cat readme.txt
# the line above shows the content of the readme file, which explains the structrue of the dataset
# Lemmatisation is a process similar to stemming

/content/drive/My Drive/BigData/data/lingspam_public
This directory contains the Ling-Spam corpus, as described in the 
paper:

I. Androutsopoulos, J. Koutsias, K.V. Chandrinos, George Paliouras, 
and C.D. Spyropoulos, "An Evaluation of Naive Bayesian Anti-Spam 
Filtering". In Potamias, G., Moustakis, V. and van Someren, M. (Eds.), 
Proceedings of the Workshop on Machine Learning in the New Information 
Age, 11th European Conference on Machine Learning (ECML 2000), 
Barcelona, Spain, pp. 9-17, 2000.

There are four subdirectories, corresponding to four versions of 
the corpus:

bare: Lemmatiser disabled, stop-list disabled.
lemm: Lemmatiser enabled, stop-list disabled.
lemm_stop: Lemmatiser enabled, stop-list enabled.
stop: Lemmatiser disabled, stop-list enabled.

Each one of these 4 directories contains 10 subdirectories (part1, 
..., part10). These correspond to the 10 partitions of the corpus 
that were used in the 10-fold experiments. In each repetition, one 
part was reserved for 

## Task 1) Read the dataset and create RDDs 
a) Start by reading the directory with text files from the file system (`/content/drive/My Drive/BigData/data/lingspam_public`). Load all text files per dirctory (part1,part2, ... ,part10) using `wholeTextFiles()`, which creates one RDD per part, containing tuples (filename,text). This is a good choice as the text files are small. (5%)

b) We will use one of the RDDs as test set, the rest as training set. For the training set you need to create the union of the remaining RDDs. (5%)

b) Remove the path and extension from the filename using the regular expression provided (5%).

If the filename starts with 'spmsg' it is spam, otherwise it is not. We'll use that later to train a classifier. 

We will put the code in each cell into a function that we can reuse later. In this way we can develop the whole preprocessing with the smaller test set and apply it to the training set once we know that everything works. 

In [0]:
from pathlib import Path
import re

def makeTestTrainRDDs(pathString):
    """ Takes one of the four subdirectories of the lingspam dataset and returns two RDDs one each for testing and training. """
    # We should see10 parts that we can use for creating train and test sets.
    p = Path(pathString) # gets a path object representing the current directory path.
    dirs = list(p.iterdir()) # get the directories part1 ... part10. 
    print(dirs) # Print to check that you have the right directory. You can comment this out when checked. 
    rddList = [] # create a list for the RDDs
    # now create an RDD for each 'part' directory and add them to rddList
    for d in dirs: # iterate through the directories
    # >>>     rdd = ... #>>> # read the files in the directory 
    # >>> ... #>>> append the RDD to the rddList
      
        rdd = sc.wholeTextFiles(str(d.absolute()))
        rddList.append(rdd) #>>> append the RDD to the rddList
    print('len(rddList)',len(rddList))  # we should now have 10 RDDs in the list # just for testing
    print(rddList[1].take(1)) # just for testing, comment out when it works.
    testRDD1 = rddList[9] # set the test set
    trainRDD1 = rddList[0] # start the training set from 0 and 
    # now loop over the range from 1 to 9(exclusive) to create a union of the remaining RDDs
    for i in range(1,9):
        trainRDD1 = trainRDD1.union(rddList[i]) #>>> create a union of the current and the next 
            # RDD in the list, so that in the end we have a union of all parts 0-8. (9 ist used as test set)
    # both RDDs should remove the paths and extensions from the filename. 
    #>>> This regular expression will do it: re.split('[/\.]', fn_txt[0])[-2]
    #>>> apply it to the filenames in train and test RDD with a lambda
    testRDD2 = testRDD1.map(lambda fn_txt: (re.split('[/\.]', fn_txt[0])[-2], fn_txt[1]))
    trainRDD2 = trainRDD1.map(lambda fn_txt: (re.split('[/\.]', fn_txt[0])[-2], fn_txt[1])) 
    return (trainRDD2,testRDD2)

# this makes sure we are in the right directory
%cd /content/drive/My Drive/BigData/data/lingspam_public 
# this should show "bare  lemm  lemm_stop  readme.txt  stop"
!ls
# the code below is for testing the function makeTestTrainRDDs
trainRDD_testRDD = makeTestTrainRDDs('bare') # read from the 'bare' directory - this takes a bit of time
(trainRDD,testRDD) = trainRDD_testRDD # unpack the returned tuple
print('created the RDDs') # notify the user, so that we can figure out where things went wrong if they do.
print('testRDD.count(): ',testRDD.count()) # should be ~291 
#print('trainRDD.count(): ',trainRDD.count()) # should be ~2602 - commented out to save time as it takes some time to create RDD from all the files
print('testRDD.getNumPartitions()',testRDD.getNumPartitions()) # normally 2 on Colab (single machine)
print('testRDD.getStorageLevel()',testRDD.getStorageLevel()) # Serialized, 1x Replicated 
print('testRDD.take(1): ',testRDD.take(1)) # should be (filename,[tokens]) 
rdd1 = testRDD # use this for developemnt in the next tasks 

/content/drive/My Drive/BigData/data/lingspam_public
bare  lemm  lemm_stop  readme.txt  stop
[PosixPath('bare/part10'), PosixPath('bare/part9'), PosixPath('bare/part8'), PosixPath('bare/part7'), PosixPath('bare/part6'), PosixPath('bare/part5'), PosixPath('bare/part4'), PosixPath('bare/part3'), PosixPath('bare/part2'), PosixPath('bare/part1')]
('len(rddList)', 10)
[(u'file:/content/drive/My Drive/BigData/data/lingspam_public/bare/part9/8-817msg1.txt', u'Subject: disc : grammar in uk schools\n\ni was disturbed to find some traditional fallacies in geoffrey sampson \'s discussion of the teaching of grammar in schools . though i no longer have a copy of prof . cameron \'s original post , i do recall the essentials of it , and found that mr . sampson had passed over the valid point it was making in favour of a prescriptivist , " back to basics " defense of traditional grammatical education . prof . cameron is perfectly correct in ridiculing the inflexible , rote and prescriptive approach to

## Task 2) Tokenize and remove punctuation

Now we need to split the words, a process called *tokenization* by linguists, and remove punctuation. 

We will use the Python [Natural Language Toolkit](http://www.nltk.org) *NLTK* to do the tokenization (rather than splitting ourselves, as these specialist tools usually do that better than we can ourselves). We use the NLTK function word_tokenize, see here for a code example: [http://www.nltk.org/book/ch03.html](http://www.nltk.org/book/ch03.html). 5%

Then we will remove punctuation. There is no specific funtion for this, so we use a regular expression (see here for info [https://docs.python.org/3/library/re.html?highlight=re#module-re](https://docs.python.org/3/library/re.html?highlight=re#module-re)) in a list comprehension (here's a nice visual explanation: [http://treyhunner.com/2015/12/python-list-comprehensions-now-in-color/](http://treyhunner.com/2015/12/python-list-comprehensions-now-in-color/)). 5% 

We use a new technique here: we separate keys and values of the RDD, using the RDD functions `keys()` and `values()`, which yield each a new RDD. Then we process the values and *zip* them together with the keys again. See here for documentation: [http://spark.apache.org/docs/2.4.0/api/python/pyspark.html#pyspark.RDD.zip](http://spark.apache.org/docs/2.4.0/api/python/pyspark.html#pyspark.RDD.zip).  We wrap the whole sequence into one function `prepareTokenRDD` for later use. 5%

In [0]:
import nltk
import re
from nltk.corpus import stopwords

def tokenize(text):
    """ Apply the nltk.word_tokenize() method to our text, return the token list. """
    nltk.download('punkt') # this loads the standard NLTK tokenizer model 
    # it is important that this is done here in the function, as it needs to be done on every worker.
    # If we do the download outside a this function, it would only be executed on the driver     
## >>>    return ... # use the nltk function word_tokenize
    return  nltk.word_tokenize(text)

def removePunctuation(tokens):
    """ Remove punctuation characters from all tokens in a provided list. """
    # this will remove all punctiation from string s: re.sub('[()\[\],.?!";_]','',s)
##>>>  tokens2 =  [...] # use a list comprehension to remove punctuaton
    tokens2 = [re.sub('[()\[\],.?!";_]','',s) for s in tokens]
    return tokens2 
def prepareTokenRDD(fn_txt_RDD):
    """ Take an RDD with (filename,text) elements and transform it into a (filename,[token ...]) RDD without punctuation characters. """
    rdd_vals2 = fn_txt_RDD.values() # It's convenient to process only the values. 
    rdd_vals3 = rdd_vals2.map(tokenize) # Create a tokenised version of the values by mapping
    rdd_vals4 = rdd_vals3.map(removePunctuation) # remove punctuation from the values
    rdd_kv = fn_txt_RDD.keys().zip(rdd_vals4) # we zip the two RDDs together 
    # i.e. produce tuples with one itme from each RDD.
    # This works because we have only applied mappings to the values, 
    # therefore the items in both RDDs are still aligned.
    # now remove any empty value strings (i.e. length 0) that we may have created by removing punctiation.
    # >>> now remove any empty strings (i.e. length 0) that we may have 
    # created by removing punctuation, and resulting entries without words left.
    rdd_kvr = rdd_kv.map(lambda x: (x[0], [s for s in x[1] if len(s)>0])) #empty strings using RDD.map and a lambda. TIP len(s) gives you the lenght of string. 
    rdd_kvrf = rdd_kvr.filter(lambda x: len(x[1])>0) # remove items without tokens using RDD.filter and a lambda. 
    # >>> Question: why should this be filtering done after zipping the keys and values together?
    # For preservation of mapping between values and their tokens, keys and values have to be zipped together before filtering occurs. 
    return rdd_kvrf

rdd2 = prepareTokenRDD(rdd1) # Use a small RDD for testing.
print(rdd2.take(1)) # For checking result of task 2. 

[(u'3-1msg1', [u'Subject', u':', u're', u':', u'2', u'882', u's', u'-', u'>', u'np', u'np', u'>', u'date', u':', u'sun', u'15', u'dec', u'91', u'02', u':', u'25', u':', u'02', u'est', u'>', u'from', u':', u'michael', u'<', u'mmorse', u'@', u'vm1', u'yorku', u'ca', u'>', u'>', u'subject', u':', u're', u':', u'2', u'864', u'queries', u'>', u'>', u'wlodek', u'zadrozny', u'asks', u'if', u'there', u'is', u'``', u'anything', u'interesting', u'``', u'to', u'be', u'said', u'>', u'about', u'the', u'construction', u'``', u's', u'>', u'np', u'np', u'``', u'second', u'>', u'and', u'very', u'much', u'related', u':', u'might', u'we', u'consider', u'the', u'construction', u'to', u'be', u'a', u'form', u'>', u'of', u'what', u'has', u'been', u'discussed', u'on', u'this', u'list', u'of', u'late', u'as', u'reduplication', u'the', u'>', u'logical', u'sense', u'of', u'``', u'john', u'mcnamara', u'the', u'name', u'``', u'is', u'tautologous', u'and', u'thus', u'at', u'>', u'that', u'level', u'indistinguishabl

^ For preservation of mapping between values and their tokens, keys and values have to be zipped together before filtering occurs.

## Task 3) Creating normalised TF.IDF vectors of defined dimensionality, measure the effect of caching.

We use the hashing trick to create fixed size TF vectors directly from the word list now (slightly different from the previous lab, where we used *(word,count)* pairs.). Write a bit of code as needed. (5%)

Then we'll use the IDF and Normalizer functions provided by Spark. They use a slightly different pattern than RDD.map and reduce, have a look at the examples here in the documentation for Normalizer  and IDF:
[http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.feature.Normalizer](http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.feature.Normalizer), [http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.feature.IDF](http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.feature.IDF) (5%)

We want control of the dimensionality in the `normTFIDF` function, so we introduce an argument into our functions that enables us to vary dimensionalty later. Here is also an opportunity to benefit from caching, i.e. persisting the RDD after use, so that it will not be recomputed.  (5%)

In [0]:
from pyspark import StorageLevel

# use the hashing trick to create a fixed-size vector from a word list
def hashing_vectorize(text,N): # arguments: the list and the size of the output vector
    v = [0] * N  # create vector of 0s
    for word in text: # iterate through the words 
#>>>              # get the hash value 
#>>>              # add 1 at the hashed address 
                   h = hash(word)
                   v[h % N] = v[h % N] + 1
    return v # return hashed word vector

from pyspark.mllib.feature import IDF, Normalizer

def normTFIDF(fn_tokens_RDD, vecDim, caching=True):
    keysRDD = fn_tokens_RDD.keys()
    tokensRDD = fn_tokens_RDD.values()
    tfVecRDD = tokensRDD.map(lambda tokens: hashing_vectorize(tokens,vecDim)) #>>> passing the vecDim value. TIP: you need a lambda. 
    if caching:
        tfVecRDD.persist(StorageLevel.MEMORY_ONLY) # since we will read more than once, caching in Memory will make things quicker.
    idf = IDF() # create IDF object
    idfModel = idf.fit(tfVecRDD) # calculate IDF values
    tfIdfRDD = idfModel.transform(tfVecRDD) # 2nd pass needed (see lecture slides), transforms RDD
#>>>    norm = ... # create a Normalizer object like in the example linked above
#>>>    normTfIdfRDD = norm. ... # and apply it to the tfIdfRDD 
#>>>    zippedRDD = ... # zip the keys and values together
    norm = Normalizer()
    normTfIdfRDD = norm.transform(tfIdfRDD)
    zippedRDD = keysRDD.zip(normTfIdfRDD)
    return zippedRDD

testDim = 10 # too small for good accuracy, but OK for testing
rdd3 = normTFIDF(rdd2, testDim, True) # test our
print(rdd3.take(1)) # we should now have tuples with ('filename',[N-dim vector])
# e.g. [('9-1142msg1', DenseVector([0.0, 0.0, 0.0, 0.0, 0.4097, 0.0, 0.0, 0.0, 0.9122, 0.0]))]

[(u'3-1msg1', DenseVector([0.5982, 0.0, 0.2114, 0.1057, 0.0, 0.0, 0.5678, 0.0, 0.5137, 0.0]))]


### Task 3a) Caching experiment

The normTFIDF lets us switch caching on or off. Write a bit of code that measures the effect of caching by takes the time for both options. Use the time function as shown in lecture 3, slide 47. Remember that you need to call an action on an RDD to trigger full execution. 

Add a short comment on the result (why is there an effect, why of the size that it is?). Remember that this is wall clock time, i.e. you may get noisy results and they may change depending on the system state (e.g. how often this test has been run). (10%)

In [0]:
#run a small experiment with caching set to True or False, 3 times each

from time import time

resCaching = [] # for storing results
resNoCache = [] # for storing results
for i in range(3): # 3 samples
#>>>  # start timer
    startTime = time()
    testRDD1 = normTFIDF(rdd2, testDim, True) # 
#>>>     # call an action on the RDD to trigger execution
#>>>  # end timer
    testRDD1.collect()
    endTime = time()
    resCaching.append( endTime - startTime ) # calculate the time spent
    
for i in range(3): # 3 samples
#>>>  # start timer
    startTime = time()
    testRDD2 = normTFIDF(rdd2, testDim, False) 
#>>>  # call an action on the RDD to trigger execution   
#>>>  # end timer
    testRDD2.collect()
    endTime = time()
    resNoCache.append( endTime - startTime ) # calculate the time spent
    
#>>> meanTimeCaching = # calculate average times
#>>> meanTimeNoCache = # calculate average times
meanTimeCaching = sum(resCaching)/len(resCaching)
meanTimeNoCache = sum(resNoCache)/len(resNoCache)
print('Creating TF.IDF vectors, 3 trials - mean time with caching: ', meanTimeCaching, ', mean time without caching: ', meanTimeNoCache)
#>>> # add your results and comments here 

('Creating TF.IDF vectors, 3 trials - mean time with caching: ', 18.616979281107586, ', mean time without caching: ', 24.16119869550069)


The two mean times are; 
1) mean time with caching: 18.616979281107586
2) mean time without caching: 24.16119869550069  

Caching an RDD is a quicker way of giving results without repeating the same actions on each RDD, it is like storing the result instead of re-calculating. Since the average time of non-cached RDD is higher, we can infer from this that wall clock times are accurate and precision level is adequate. If the opposite was true, this would mean that since our Data is textual there is a larger amount of additional meaningless information also the data could have not been understand by the machine because it is unstrcuctured. Increasing the size of the sample would result in an increased difference between the two averages. Also increaseing size of RDD would result in an increased effect of caching times. 


## Task 4) Create LabeledPoints 

Determine whether the file is spam (i.e. the filename contains ’spmsg’) and replace the filename by a 1 (spam) or 0 (non-spam) accordingly. Use `RDD.map()` to create an RDD of LabeledPoint objects. See here [http://spark.apache.org/docs/2.1.0/mllib-linear-methods.html#logistic-regression](http://spark.apache.org/docs/2.1.0/mllib-linear-methods.html#logistic-regression) for an example, and here [http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.regression.LabeledPoint](http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.regression.LabeledPoint) for the `LabeledPoint` documentation. (10%)

There is a handy function of Python strings called startswith: e.g. 'abc'.startswith('ab) will return true. The relevant Python syntax here is a conditional expression: **``<a> if <yourCondition> else <b>``**, i.e. 1 if the filename starts with 'spmsg' and otherwise 0.

In [0]:
from pyspark.mllib.regression import LabeledPoint

# creatate labelled points of vector size N out of an RDD with normalised (filename [(word,count), ...]) items
def makeLabeledPoints(fn_vec_RDD): # RDD and N needed 
    # we determine the true class as encoded in the filename and represent as 1 (samp) or 0 (good)
#>>>    cls_vec_RDD = fn_vec_RDD.map(lambda ... ) # use a conditional expression to get the class label (True or False)
    cls_vec_RDD = fn_vec_RDD.map(lambda x: [1 if x[0].startswith('spmsg') == True else 0, x[1]])
    # now we can create the LabeledPoint objects with (class,vector) arguments
    lp_RDD = cls_vec_RDD.map(lambda cls_vec: LabeledPoint(cls_vec[0],cls_vec[1]) ) 
    return lp_RDD 

# for testing
testLpRDD = makeLabeledPoints(rdd3) 
print(testLpRDD.take(1)) 
# should look similar to this: [LabeledPoint(0.0, [0.0,0.0,0.0,0.0,0.40968062880166006,0.0,0.0,0.0,0.9122290186048,0.0])]

[LabeledPoint(0.0, [0.5982438084217956,0.0,0.21143411637490286,0.10571705818745143,0.0,0.0,0.5677603124560466,0.0,0.5136848171065144,0.0])]


## Task 5) Complete the preprocessing 

It will be useful to have a single function to do the preprocessing. So integrate everything here. (5%)

In [0]:
# now we can apply the preprocessing chain to the data loaded in task 1 
# N is for controlling the vector size
def preprocess(rawRDD,N):
    """ take a (filename,text) RDD and transform into LabelledPoint objects 
        with class labels and a TF.IDF vector with N dimensions. 
    """
    tokenRDD = prepareTokenRDD(rawRDD) # task 2
    tfIdfRDD = normTFIDF(tokenRDD,N) # task 3
    lpRDD = makeLabeledPoints(tfIdfRDD) # task 4
    return lpRDD # return RDD with LabeledPoints

# and with this we can start the whole process from a directory, N is again the vector size
def loadAndPreprocess(directory,N):
    """ load lingspam data from a directory and create a training and test set of preprocessed data """
#>>>    trainRDD_testRDD = ... # read from the directory using the function created in task 1
#>>>     # unpack the returned tuple
    trainRDD_testRDD = makeTestTrainRDDs(directory)
    (trainRDD,testRDD) = trainRDD_testRDD

    return (preprocess(trainRDD,N),preprocess(testRDD,N)) # apply the preprocessing funcion defined above

trainLpRDD = preprocess(trainRDD,testDim) # prepare the training data
print(testLpRDD.take(1)) # should look similar to previous cell's output

train_test_LpRDD = loadAndPreprocess('lemm',100) # let's re-run with another vector size
(trainLpRDD,testLpRDD) = train_test_LpRDD 
print(testLpRDD.take(1))
print(trainLpRDD.take(1)) 

[LabeledPoint(0.0, [0.5982438084217956,0.0,0.21143411637490286,0.10571705818745143,0.0,0.0,0.5677603124560466,0.0,0.5136848171065144,0.0])]
[PosixPath('lemm/part10'), PosixPath('lemm/part9'), PosixPath('lemm/part8'), PosixPath('lemm/part7'), PosixPath('lemm/part6'), PosixPath('lemm/part5'), PosixPath('lemm/part4'), PosixPath('lemm/part3'), PosixPath('lemm/part2'), PosixPath('lemm/part1')]
('len(rddList)', 10)
[(u'file:/content/drive/My Drive/BigData/data/lingspam_public/lemm/part9/8-829msg1.txt', u"Subject: re : 8 . 810 , sum : french loan word , language evolution\n\njust see your summary on french loanword in english . i think baugh take the 10 , 0 word figure from jespersen 's _ growth and structure of the english language _ . it would be more like jespersen than baugh to actually try to count them , in fact . it 's worth look at jespersen for all sort of fact like these , in his chapter on the french influence in that book . - - suzanne kemmer\n")]
[LabeledPoint(0.0, [0.0,0.3373374

## Task 6) Train some classifiers 

Use the `LabeledPoint` objects to train a classifier, specifically the *LogisticRegression*, *Naive Bayes*, and *Support Vector Machine*. Calculate the accuracy of the model on the training set (again, follow this example [http://spark.apache.org/docs/2.1.0/ml-classification-regression.html#logistic-regression](http://spark.apache.org/docs/2.0.0/ml-classification-regression.html#logistic-regression) and here is the documentation for the classifiers [LogisticRegressionWithLBFGS](http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.classification.LogisticRegressionWithLBFGS), [NaiveBayes](http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.classification.NaiveBayes), [SVMWithSGD](http://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html#pyspark.mllib.classification.SVMWithSGD).  (10%) 

In [0]:
from pyspark.mllib.classification import (NaiveBayes, LogisticRegressionWithLBFGS, SVMWithSGD) 
import numpy

# train the model with an (f,[(w,c), ...]) RDD. This is practical as we can reuse the function for TF.IDF
def trainModel(lpRDD):
    """ Train 3 classifier models on the given RDD with LabeledPoint objects. A list of trained model is returned. """
    lpRDD.persist(StorageLevel.MEMORY_ONLY) # not really needed as the Spark implementations ensure caching themselves. 
                    # Other implementations might not, however. 
    # Train a classifier model.
    print('Starting to train the model') # give some immediate feedback
    model1 = LogisticRegressionWithLBFGS.train(lpRDD) # this is the best model
    print('Trained LR (model1)')
    #print('type(model1)')
    model2 = NaiveBayes.train(lpRDD) # doesn't work well
    print('Trained NB (model2)')
    #print(type(model2))
    model3 = SVMWithSGD.train(lpRDD) # or this ...
    print('Trained SVM (model3)')
    return [model1,model2,model3]

def testModel(model, lpRDD):
    """ Tests the classificatio accuracy of the given model on the given RDD with LabeledPoint objects. """
    lpRDD.persist(StorageLevel.MEMORY_ONLY)
    # Make prediction and evaluate training set accuracy.
    # Get the prediction and the ground truth label
    predictionAndLabel = lpRDD.map(lambda p: (model.predict(p.features), p.label)) # get the prediction and ground truth (label) for each item.
    correct = predictionAndLabel.filter(lambda xv: xv[0] == xv[1]).count() # count the correct predictions 
#>>>    accuracy = ... # and calculate the accuracy 
    accuracy = float(correct)/float(lpRDD.count())
    print('Accuracy {:.1%} (data items: {}, correct: {})'.format(accuracy,lpRDD.count(), correct)) # report to console
    return accuracy # and return the value  

models = trainModel(trainLpRDD) # just for testing
testModel(models[2], trainLpRDD) # just for testing

Starting to train the model
Trained LR (model1)
Trained NB (model2)
Trained SVM (model3)
Accuracy 83.4% (data items: 2604, correct: 2171)


0.8337173579109063

## Task 7) Automate training and testing

We automate now the whole process from reading the files, through preprocessing, and training up to evaluating the models. In the end we have a single function that takes all the parameters we are interested in and produces trained models and an evaluation. (5%) 

In [0]:
# this function combines tasks f) and g)
# this method should take RDDs with (f,[(w,c), ...])
def trainTestModel(trainRDD,testRDD):
    """ Trains 3 models and tests them on training and test data. Returns a matrix the training and testing (rows) accuracy values for all models (columns). """
#>>> models = ... # train models on the training set
    models = trainModel(trainRDD)
    results = [[],[]] # matrix for 2 modes (training/test) vs n models (currently 3)
    for mdl in models:
        print('Training')
#>>>     results[0].append(...) # test the model on the training set
        results[0].append(testModel(mdl,trainRDD))
    
        print('Testing')
#>>>     #results[1].append(...) # test the model on the test set
        results[1].append(testModel(mdl,testRDD))
    return results

def trainTestFolder(folder,N):
    """ Reads data from a folder, preproceses the data, and trains and evaluates models on it. """
    print('Start loading and preprocessing') 
    train_test_LpRDD = loadAndPreprocess(folder,N) # create the RDDs
    print('Finished loading and preprocessing')
    (trainLpRDD,testLpRDD) = train_test_LpRDD # unpack the RDDs 
    return trainTestModel(trainLpRDD,testLpRDD) # train and test

trainTestFolder('lemm',1000) 

Start loading and preprocessing
[PosixPath('lemm/part10'), PosixPath('lemm/part9'), PosixPath('lemm/part8'), PosixPath('lemm/part7'), PosixPath('lemm/part6'), PosixPath('lemm/part5'), PosixPath('lemm/part4'), PosixPath('lemm/part3'), PosixPath('lemm/part2'), PosixPath('lemm/part1')]
('len(rddList)', 10)
[(u'file:/content/drive/My Drive/BigData/data/lingspam_public/lemm/part9/8-829msg1.txt', u"Subject: re : 8 . 810 , sum : french loan word , language evolution\n\njust see your summary on french loanword in english . i think baugh take the 10 , 0 word figure from jespersen 's _ growth and structure of the english language _ . it would be more like jespersen than baugh to actually try to count them , in fact . it 's worth look at jespersen for all sort of fact like these , in his chapter on the french influence in that book . - - suzanne kemmer\n")]
Finished loading and preprocessing
Starting to train the model
Trained LR (model1)
Trained NB (model2)
Trained SVM (model3)
Training
Accuracy

[[1.0, 0.9404761904761905, 0.8337173579109063],
 [0.9688581314878892, 0.9273356401384083, 0.8339100346020761]]

## Task 8) Run experiments 

We have now a single function that allows us to vary the vector size easily. Test vector sizes 3, 30, 300, 3000, 30000 and examine the effect on the classification accuracy in Experiment 1.

Use the function from Task 7) to test different data types. The dataset has raw text in folder `bare`, lemmatised text in  `lemm` (similar to stemming, reduces to basic word forms), `stop` (with stopwords removed), and `lemm_stop` (lemmatised and stopwords removed). Test how the classification accuracy differs for these four data types in Experiment 2. Collect the results in a data structure that can be saved for later saving and analyis.

Comment on the results in a few sentences, considering the differences in performance between the different conditions as well as train an test values. 15%

In [0]:
from pyspark.sql import DataFrame

folder = 'bare'
N = numpy.array([3,30,300,3000,30000]) 
print('\nEXPERIMENT 1: Testing different vector sizes')
results = []
for n in N:
    print('N = {}'.format(n))
    result = {'n':n, 't':folder}
    result['acc'] = trainTestFolder(folder,n)
    results.append(result)

    
n = 3000
typeFolders = ['bare','stop','lemm','lemm_stop']
print('EXPERIMENT 2: Testing different data types')
results2 = []
for folder in typeFolders:
    print('Path = {}'.format(folder))
    result = {'n':n, 't':folder}
    result['acc'] = trainTestFolder(folder,n)
    results2.append(result)

# Add comments on the performance in a cell below. 


EXPERIMENT 1: Testing different vector sizes
N = 3
Start loading and preprocessing
[PosixPath('bare/part10'), PosixPath('bare/part9'), PosixPath('bare/part8'), PosixPath('bare/part7'), PosixPath('bare/part6'), PosixPath('bare/part5'), PosixPath('bare/part4'), PosixPath('bare/part3'), PosixPath('bare/part2'), PosixPath('bare/part1')]
('len(rddList)', 10)
[(u'file:/content/drive/My Drive/BigData/data/lingspam_public/bare/part9/8-817msg1.txt', u'Subject: disc : grammar in uk schools\n\ni was disturbed to find some traditional fallacies in geoffrey sampson \'s discussion of the teaching of grammar in schools . though i no longer have a copy of prof . cameron \'s original post , i do recall the essentials of it , and found that mr . sampson had passed over the valid point it was making in favour of a prescriptivist , " back to basics " defense of traditional grammatical education . prof . cameron is perfectly correct in ridiculing the inflexible , rote and prescriptive approach to grammar 

##Logistic Regression:  
Increasing the vector size results in an increase in both the testing and training accuracy. For n = 30 000, LR reaches an impressive 100% for training accuracy.  

### Naive Bayes: 
An increasing in vector size yields an increase in both trianing and testing accuracey, at n = 30000, training accurecy is at 86.8%. 

##SVM: 
SVM seems to be uneffected by vector size; at n = 3 and n = 30000, both training and testing accuracy is at 83.4%. 

## Model Performance: 

Logistic Regression outperformes Naive Bayes and SVM for allmost all data sizes, i.e. n and for different experients. Also for each model, training accuracy is higher than testing accuracy. at n = 3, all three model's training and testing accuracy is at 83.4%. The rule is; an increase in vector size should give an increase in classification accuracey which proofs to be the case above, this is due to the effect of hashing. 

## Vector Size: 
An increase in vector size should yield an increase in training/testing accuracy as hashing maps each word in text to a positin in a vector which leads to data being spread out and hence collisions become less fequent. collisions should be avoided as they cause misclassifications. 

###Data Size 
Classifiers can yield very accurate results if more data is used. Banko and Brill (2001) proof that having less  than one million words in a text can cause greater variation in the performance of classifiers, this sheds light on why SVM's accuracy did not improve and on the difference between LR and NB.  
